In [89]:
import gurobipy as gp
from gurobipy import *
from gurobipy import GRB

## K-Satisfiability problem (3-SAT)
- n개의 이진 변수 (Literal), K개의 literal로 이루어진 논리합 (clause), clause들의 논리 곱으로 표현된 논리곱 표준형(CNF)의 충족 가능성을 찾는 문제
- 3-SAT는 최대 3개의 literal로 이루어진 clause를 지닌 K-SAT 문제, 모든 clause가 3개의 literal로 이루어져 있다면 exact-3-SAT 문제
- 어찌보면 목적함수가 없고, Constraints를 만족하는 해가 하나 이상 있는지를 판단하는 최적화 문제라고도 볼 수 있음

## Problem info description
- uf20/uf50/uf75
  - Number of literal: 20/50/75 
  - Number of caluses: 91/218/325
  - All problems solution == True, Exact-3-SAT
- uuf20/uuf50/uuf75
  - Number of literal: 20/50/75 
  - Number of caluses: 91/218/325
  - All problems solution == False, Exact-3-SAT
- Horn clause
  - clause 내의 literal이 적어도 하나 이상 1인 문제
- Forced generation
  - SAT를 만족하는 문제를 생성하기 위해 생성 프로세스의 첫 clause를 랜덤하게 결정하고 그 다음 clause 부터는 첫 clause에 연쇄적으로 문제를 만드는 경우 --> SLS 알고리즘에서 매우 쉽게 풀린다고 함
- Mixed SAT Problem
  - 연산자가 단일의 연산자가 아닐 수 있는 문제. 즉, CNF가 아닌 문제
  
## Problem reduction 3SAT->ILP
- 아래의 식과 같이 SAT에서 논리로 표현된 constraint를 0-1 ILP에 맞추어 변형할 수 있음 
$$ \text{clause of SAT}: x_1 \vee \overline{x_2} \vee \cdots \vee \overline{x_{n-1}} \vee x_n  $$
$$ \text{clause of 0-1 ILP}: x_1 + (1 - x_2) + \cdots + (1 - x_{n-1}) + x_n \ge 1 $$
- 이때, feasible이 하나 이상인 해를 찾는 것이기에 objective가 중요하지 않지만, 여러 개의 feasible solution 중에서도 0인 literal이 가장 많은 해가 비용이 적은 해일 것이라고 가정해서 목적함수는 아래와 같이 정의
$$ \min{Z} = \sum \limits _{i} ^{n} x_i $$

In [103]:
# insert the "SAT problem file name.cnf"
file_name = "uf50-01.cnf"
print("problem file :", file_name)
file = open(file_name, 'r')

for _ in range(2):
    file.readline()
    
isHorn = file.readline().split("c")[1].strip()
if isHorn[-1] == "o" and isHorn[-2] == "n":
    isHorn = False
else:
    isHorn = True
    
print("Horn :", isHorn)
    
isForced = file.readline().split("c")[1].strip()
if isForced[-1] == "o" and isForced[-2] == "n":
    isForced = False
else:
    isForced = True
    
print("Forced :", isForced)
    
isMixed = file.readline().split("c")[1].strip()
if isMixed[-1] == "o" and isMixed[-2] == "n":
    isMixed = False
else:
    isMixed = True
    
print("Mixed :", isMixed)
    
clauseLength = int(file.readline().split("=")[1].strip())
print("Length of Cluase :", clauseLength)
    
file.readline()

cnfInfo = file.readline().split("cnf")[1].strip()
variables = ""
clause = ""
for i in cnfInfo:
    if i == " " :
        break
    else :
        variables += i

print("Number of variables :", variables)
        
for i in cnfInfo[::-1]:
    if i == " " :
        break
    else :
        clause += i
    clause = clause[::-1]

print("Number of maximum caluses :", clause)

clause_list = []

while 1:
    current_clause = file.readline().strip()
    if current_clause == "%":
        break
    tmp_clause = []
    tmp_var = ""
    for i in current_clause:
        if i != " ":
            tmp_var += str(i)
        if i == " ":
            tmp_clause.append(tmp_var)
            tmp_var = ""
            
    clause_list.append(tmp_clause)
    
print(clause_list)
    
# result = file.readline().strip()
# if result == "0" or result == 0:
#     result = False
# else:
#     result = True

# print("Circuit result :", result)

if file_name[0] == "u" and file_name[1] == "u":
    result = False
else:
    result = True
    
print("result: ", result)
    
file.close()

problem file : uf50-01.cnf
Horn : False
Forced : True
Mixed : False
Length of Cluase : 3
Number of variables : 50
Number of maximum caluses : 281
[['-3', '36', '7'], ['-3', '-42', '-48'], ['-49', '-47', '-41'], ['8', '-40', '17'], ['-21', '-31', '-39'], ['36', '-22', '49'], ['27', '38', '14'], ['15', '-18', '6'], ['6', '7', '-43'], ['34', '-7', '23'], ['2', '14', '-13'], ['2', '47', '-42'], ['-33', '-35', '3'], ['44', '40', '49'], ['50', '36', '31'], ['-36', '-3', '-37'], ['26', '-29', '43'], ['15', '29', '-45'], ['24', '-11', '18'], ['-47', '-26', '6'], ['-50', '-33', '-10'], ['32', '6', '16'], ['-34', '37', '41'], ['7', '-28', '-17'], ['-44', '46', '19'], ['7', '22', '-48'], ['3', '39', '34'], ['31', '46', '-43'], ['-27', '32', '23'], ['37', '-50', '-18'], ['20', '5', '11'], ['-45', '-24', '6'], ['-34', '-23', '-14'], ['-22', '21', '20'], ['-17', '50', '24'], ['-25', '-24', '-27'], ['3', '35', '21'], ['-26', '47', '-36'], ['-28', '-45', '49'], ['-21', '-6', '12'], ['-17', '-15', '-39

In [104]:
model = gp.Model()
x = model.addVars(range(1, int(variables)+1), vtype=GRB.BINARY, name="x")
obj = gp.LinExpr()

for i in range(1, int(variables)+1):
    obj += x[i]
    # model.addConstr(x[i] >= 0)
    # model.addConstr(x[i] <= 1)

for [i, j, k] in clause_list:
    i, j, k = int(i), int(j), int(k)
    
    if i <= 0:
        i = -i
        tmp_i = (1-x[i])
    else:
        tmp_i = x[i]
        
    if j <= 0:
        j = -j
        tmp_j = (1-x[j])
    else:
        tmp_j = x[j]
        
    if k <= 0:
        k = -k
        tmp_k = (1-x[k])
    else:
        tmp_k = x[k]
    model.addConstr(tmp_i + tmp_j + tmp_k >= 1)
    # obj += (tmp_i + tmp_j + tmp_k)
    
model.setObjective(obj, GRB.MINIMIZE)

In [105]:
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 218 rows, 50 columns and 654 nonzeros
Model fingerprint: 0xb3733bc1
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 217 rows, 50 columns, 649 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)
Found heuristic solution: objective 19.0000000

Root relaxation: objective 1.276190e+01, 82 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   12.76190    0   38   19.00000   12.76190  32.8%     -    0s
     0     0   13.75000    0   45   19.0000

In [106]:
sol = []
try:
    for v in model.getVars():
        print(v.varName, v.x)
        sol.append(int(v.x))
    print("\n"+"objective value:", model.objVal)
except:
    print("모든 해가 infeasible 하므로 충족 불가능(False)합니다.")

x[1] 0.0
x[2] 1.0
x[3] 0.0
x[4] 1.0
x[5] 1.0
x[6] 1.0
x[7] 1.0
x[8] 1.0
x[9] 1.0
x[10] 0.0
x[11] 0.0
x[12] 1.0
x[13] 0.0
x[14] 1.0
x[15] 0.0
x[16] 0.0
x[17] 0.0
x[18] 0.0
x[19] 1.0
x[20] 1.0
x[21] 0.0
x[22] 0.0
x[23] 1.0
x[24] 0.0
x[25] 0.0
x[26] 0.0
x[27] 1.0
x[28] 0.0
x[29] 0.0
x[30] 0.0
x[31] 0.0
x[32] 0.0
x[33] 0.0
x[34] 0.0
x[35] 1.0
x[36] 1.0
x[37] 1.0
x[38] 0.0
x[39] 1.0
x[40] 0.0
x[41] 0.0
x[42] 0.0
x[43] 0.0
x[44] 0.0
x[45] 0.0
x[46] 0.0
x[47] 0.0
x[48] 1.0
x[49] 1.0
x[50] 0.0

objective value: 19.0


In [107]:
try:
    funck = 0
    for [i, j, k] in clause_list:
        i, j, k = int(i), int(j), int(k)

        if i <= 0:
            i = -i
            tmp_i = (1-sol[i-1])
        else:
            tmp_i = sol[i-1]

        if j <= 0:
            j = -j
            tmp_j = (1-sol[j-1])
        else:
            tmp_j = sol[j-1]

        if k <= 0:
            k = -k
            tmp_k = (1-sol[k-1])
        else:
            tmp_k = sol[k-1]

        if tmp_i + tmp_j + tmp_k < 1:
            funck += 1
        
    if funck == 0:
        print("solution is satified")
    else:
        print("solution isn't satisfied")
except:
    print("모든 해가 infeasible 하므로 충족 불가능(False)합니다.")

solution is satified
